In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

# librerías para el texto
import nltk
from nltk.corpus import stopwords
import spacy


import torch
from transformers import BertTokenizer, BertModel

import gensim
from gensim.models import Word2Vec
# utilizando Natural Language Toolkit
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# utilizando Tensorflow
# probando tensorflow.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import keras_nlp

In [2]:
imdb_es = pd.read_csv(r'../data/IMDB_spanish.csv', sep=',')

In [9]:
imdb_es

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo
...,...,...,...,...,...
49995,49995,I thought this movie did a down right good job...,Pensé que esta película hizo un buen trabajo a...,positive,positivo
49996,49996,"Bad plot, bad dialogue, bad acting, idiotic di...","Mala parcela, mal diálogo, mala actuación, dir...",negative,negativo
49997,49997,I am a Catholic taught in parochial elementary...,Soy católica enseñada en escuelas primarias pa...,negative,negativo
49998,49998,I'm going to have to disagree with the previou...,Voy a tener que estar en desacuerdo con el com...,negative,negativo


In [ ]:
# APLICADO A COLUMNAS

# Función para eliminar etiquetas HTML de una columna
def eliminar_etiquetas_html(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else x)
    return df

# Función para limpiar caracteres especiales de una columna
def limpiar_texto(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'[^\w\s.,]', '', x) if isinstance(x, str) else x)
    return df

# ------------->
# correción de funcion para limpiar caracteres especiales
def caracteres_especiales(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'[^\w\s]', ' ', x) if isinstance(x, str) else x)
    return df

def espacios_extra(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'\s+', ' ', x).strip() if isinstance(x, str) else x)
    return df
# ------------->

# Función para eliminar palabras vacías de una columna
def remove_stop_words(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    stop_words = set(stopwords.words('spanish'))
    df[col_out] = df[col_in].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]) if isinstance(x, str) else x)
    return df

def lemmatizador(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    # cargamos el modelo en espaniol
    nlp = spacy.load('es_core_news_sm')

    # aplicamos a cada fila de la columna entrada
    df[col_out] = df[col_in].apply(
        lambda x: ' '.join([token.lemma_ for token in nlp(x)]) if isinstance(x, str) else x
    )
    return df

def sentiment_token(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    conditions = [
        df[col_in] == 'positivo',
        df[col_in] == 'negativo']
    labels = [1, 0]
    df[col_out] = np.select(conditions, labels, default=np.nan)
    return df

def rating(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    conditions = [
        (df[col_in] >= 0) & (df[col_in] < 4),
        (df[col_in] >= 4) & (df[col_in] <= 6),
        (df[col_in] > 6) & (df[col_in] <= 10)]
    labels = ['Mala', 'Pasable/Normal', 'Buena']
    df[col_out] = np.select(conditions, labels, default=np.nan)
    return df


def rating_token(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    conditions = [
        (df[col_in] >= 0) & (df[col_in] <= 5),
        (df[col_in] >= 6) & (df[col_in] <= 10)]
    labels = [0, 1]
    df[col_out] = np.select(conditions, labels, default=np.nan)
    return df


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def vectorizador(texto : str) -> torch.Tensor:
    # se tokeniza la oración para convertirse en una estructura BERT
    inputs = tokenizer(texto, return_tensors='pt')

    # se desactiva el calculo del gradiente
    with torch.no_grad():
        outputs = model(**inputs)
    
    # extraemos los embeddings
    embeddings = outputs.last_hidden_state
    return embeddings


def obtener_embeddings(texto : str) -> np.array:
    # Mover el modelo a GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenización del texto y creacion de tensores
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generar embeddings con BERT sin cálculo de gradientes
    with torch.no_grad():
        outputs = model(**inputs)

    # tomar el embedding de la ultima capa
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()


def get_embedding(texto : str) -> np.array:
    # Tokenizacion y creación de tensores
    inputs = tokenizer(texto, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad(): # desactivar el calculo de gradientes
        outputs = model(**inputs)
        # extraer el embedding de [CLS] (índice 0)
        embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.squeeze().numpy()

In [19]:
imdbescopy = imdb_es

In [21]:
imdbescopy = (imdbescopy
              .pipe(sentiment_token, 'sentimiento', 'sentiemiento_token')
              .pipe(eliminar_etiquetas_html, 'review_es', 'review_es_mod')
              .pipe(caracteres_especiales, 'review_es_mod', 'review_es_mod')
              .pipe(espacios_extra, 'review_es_mod', 'review_es_mod')
              .pipe(remove_stop_words, 'review_es_mod', 'review_es_mod')
              .pipe(lemmatizador, 'review_es_mod', 'review_es_mod'))

In [23]:
imdbescopy.to_csv(r'../data/imdb_espaniol_mod.csv', sep=',', index=False)

In [28]:
def get_review_embedding(review : pd.Series, model : gensim.models.word2vec.Word2Vec): 
    word_vectors = [model.wv[word] for word in review if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [126]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential

import warnings

warnings.filterwarnings('ignore')

In [73]:
imdbesmod = pd.read_csv(r'../data/imdb_espaniol_mod.csv', sep=',')

In [74]:
imdbesmodcopy = imdbesmod[['sentiemiento_token', 'review_es_mod']]
imdbesmodcopy['sentiemiento_token'] = imdbesmodcopy['sentiemiento_token'].astype('Int64')

C:\Users\daalvarado\AppData\Local\Temp\ipykernel_38532\2524330471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdbesmodcopy['sentiemiento_token'] = imdbesmodcopy['sentiemiento_token'].astype('Int64')


In [75]:
imdbesmodcopy = imdbesmodcopy.sample(frac=1)
imdbesmodcopy_train = imdbesmodcopy.iloc[2500:]
imdbesmodcopy_test = imdbesmodcopy.iloc[:2500]

In [76]:
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(imdbesmodcopy_train['review_es_mod'])

In [86]:
# cambiando el texto a secuencias
X_train = tokenizer.texts_to_sequences(imdbesmodcopy_train['review_es_mod'])
X_test = tokenizer.texts_to_sequences(imdbesmodcopy_test['review_es_mod'])

# pad sequences
maxlen = max([len(x) for x in X_train])

print('Max Length:', maxlen)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

Max Length: 1870


In [104]:
y_train = imdbesmodcopy_train['sentiemiento_token'].values
y_test = imdbesmodcopy_test['sentiemiento_token'].values

In [88]:
embed_size = 128

In [89]:
model = Sequential([
    Embedding(max_features, embed_size),
    LSTM(60, return_sequences=True),
    GlobalMaxPool1D(),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(2, activation='sigmoid')
])

In [90]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [92]:
# entrenando el modelo
batch_size = 100
epochs = 3

In [105]:
# validacion y evaluacion
validation_split = 0.5
validation_size  =int(len(X_test) * validation_split)

X_val = X_test[:validation_size]
y_val = y_test[:validation_size]

X_eval = X_test[validation_size:]
y_eval = y_test[validation_size:]

In [94]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/3
475/475 ━━━━━━━━━━━━━━━━━━━━ 2405s 5s/step - accuracy: 0.7631 - loss: 0.4715 - val_accuracy: 0.5120 - val_loss: 1.4819
Epoch 2/3
475/475 ━━━━━━━━━━━━━━━━━━━━ 2426s 5s/step - accuracy: 0.9155 - loss: 0.2186 - val_accuracy: 0.5152 - val_loss: 1.7649
Epoch 3/3
475/475 ━━━━━━━━━━━━━━━━━━━━ 2526s 5s/step - accuracy: 0.9461 - loss: 0.1478 - val_accuracy: 0.5080 - val_loss: 1.9653


In [109]:
score = model.evaluate(X_eval, y_eval, batch_size=batch_size)

13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 595ms/step - accuracy: 0.8854 - loss: 0.3002


In [110]:
print(f'Test loss: {score[0]} - Test accuracy: {score[1]}')

Test loss: 0.3097416162490845 - Test accuracy: 0.881600022315979


### Pruebas del modelo

Modelo piloto. Este se modificará luego de hacer pruebas de rendimiento.

In [113]:
prueba = [
    "El café en este lugar es increíblemente aromático y el ambiente es perfecto para relajarse. ¡Muy recomendado!",
    "El servicio fue lento y las porciones pequeñas para el precio. No lo recomendaría.",
    "El libro tiene una trama cautivadora y personajes memorables. Me mantuvo enganchado de principio a fin.",
    "La película fue predecible y los efectos especiales parecían anticuados. Me decepcionó.",
    "Este champú dejó mi cabello suave y brillante, y el aroma es maravilloso. Definitivamente lo volveré a comprar.",
    "La aplicación se traba constantemente y consume mucha batería. No vale la pena descargarla.",
    "La chaqueta es súper cómoda y tiene un diseño moderno. Perfecta para el invierno.",
    "La comida llegó fría y con un sabor mediocre. No volveré a pedir de aquí.",
    "El curso es muy claro y los ejemplos son prácticos. Aprendí muchísimo en poco tiempo.",
    "Las instrucciones no eran claras y el producto vino defectuoso. Un desperdicio de dinero."
]
sentimiento = [
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo'
]
resenias = pd.DataFrame({'review_es': prueba, 'sentimiento': sentimiento})
resenias = resenias.sample(frac=1)

In [111]:
def preprocessing_pipe(df : pd.DataFrame, col_in : str, col_out : str) -> pd.DataFrame:
    result = (df
              .pipe(eliminar_etiquetas_html, col_in, col_out)
              .pipe(caracteres_especiales, col_out, col_out)
              .pipe(espacios_extra, col_out, col_out)
              .pipe(remove_stop_words, col_out, col_out)
              .pipe(lemmatizador, col_out, col_out))
    return result

In [116]:
resenias_treated = preprocessing_pipe(resenias, 'review_es', 'review_es_mod')

In [118]:
resenias_treated['sentimiento_token'] = resenias_treated['sentimiento'].map({'positivo': 1, 'negativo': 0})

In [119]:
resenias_treated

,review_es,sentimiento,review_es_mod,sentimiento_token
8,El curso es muy claro y los ejemplos son práct...,positivo,curso claro ejemplo práctico Aprendí muchísimo...,1
9,Las instrucciones no eran claras y el producto...,negativo,instrucción claro producto venir defectuoso de...,0
1,El servicio fue lento y las porciones pequeñas...,negativo,servicio lento porción pequeño precio recomendar,0
6,La chaqueta es súper cómoda y tiene un diseño ...,positivo,chaqueta súper cómodo diseño moderno Perfecta ...,1
3,La película fue predecible y los efectos espec...,negativo,película predecible efecto especial parecer an...,0
7,La comida llegó fría y con un sabor mediocre. ...,negativo,comida llegar fría sabor mediocre volver pedir...,0
0,El café en este lugar es increíblemente aromát...,positivo,café lugar increíblemente aromático ambiente p...,1
5,La aplicación se traba constantemente y consum...,negativo,aplicación trar constantemente consumir mucho ...,0
2,El libro tiene una trama cautivadora y persona...,positivo,libro trama cautivadora personaje memorable ma...,1
4,"Este champú dejó mi cabello suave y brillante,...",positivo,champú dejar cabello suave brillante aroma mar...,1


In [120]:
prueba_text_plain = resenias_treated['review_es_mod'].tolist()

In [125]:
prueba_text_plain_token = tokenizer.texts_to_sequences(prueba_text_plain['review_es_mod'])
prueba_text_plain_token = sequence.pad_sequences(prueba_text_plain_token, maxlen=maxlen, padding='post')

AttributeError: 'BertTokenizer' object has no attribute 'texts_to_sequences'